In [1]:
from operator import itemgetter
import json
from sklearn.metrics.pairwise import euclidean_distances
import math
from scipy.spatial import distance
import pickle
import numpy as np
from sklearn.decomposition import PCA
import pickle
from sklearn.metrics.pairwise import paired_distances
from scipy.spatial.distance import pdist, squareform

In [2]:
# def check_if_5_and_append(listobj,distance,index):
#     if distance < max(listobj,key=itemgetter(1))[1] and distance!=0:
#         listobj.pop(listobj.index(max(listobj, key=itemgetter(1))))
#         listobj.append((index,distance))
#     else:
#         pass
#     return listobj

In [2]:
with open('moma_feature_flatten.pickle', 'rb') as handle:
    feature_flatten = pickle.load(handle)
keylist = []
for item,value in feature_flatten.items():
    keylist.append(item)

In [3]:
#stack feature vectors for calculation
v0 = feature_flatten[keylist[0]].reshape(1, -1)[0]
bufferlist=[v0]
for i in range(1,len(feature_flatten)):
    v1 = feature_flatten[keylist[i]].reshape(1, -1)[0]
    bufferlist.append(v1)
    
result = np.stack(bufferlist,axis=0)

In [4]:
#perform PCA to reduce each feature vector from 25088 dimensions to 150 dimensions
pca = PCA(n_components=150,svd_solver='randomized')
result2 = pca.fit_transform(result) 

In [88]:
#after PCA, calculate distance matrix
distances = euclidean_distances(result2, result2)

In [8]:
outfile = open('dataset_pca_ed.pickle','wb')
pickle.dump(result2,outfile)
outfile.close()

In [9]:
#more strict implementation
distances2_1 = pdist(result2)

# pdist just returns the upper triangle of the pairwise distance matrix. to get
# the whole (20, 20) array we can use squareform:

print(distances2_1.shape)
# (190,)

distances2 = squareform(distances2_1)
print(distances2.shape)

(1135809291,)
(47662, 47662)


In [13]:
distances2[46000]

array([648.64765808, 715.21227506, 859.89906107, ..., 605.94898662,
       935.56196736, 654.49343316])

In [14]:
outfile = open('distances_strict.pickle','wb')
pickle.dump(distances2,outfile)
outfile.close()

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [105]:
result3=np.transpose(result2)
distances2 = paired_distances(result2,result3)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 150 while Y.shape[1] == 47662

In [101]:
result2.shape

(47662, 150)

In [104]:
np.transpose(result2).shape

(150, 47662)

In [100]:
distances2[3]

0.0

In [115]:
feature_flatten[keylist[0]][3]

4.9407907

In [113]:
len(feature_flatten[keylist[2]])

25088

In [95]:
keylist[0]

'42434'

In [87]:
distances2

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [12]:
testrow=distances[0]
print(testrow[np.argsort(testrow)[-5:]])

[1314.9171 1354.0814 1382.3148 1393.6299 2304.0598]


In [50]:
import time
start = time.time()
testrow=distances[0]
indices = np.argsort(testrow)[1:6]
result = testrow[indices]
bufferlist = [{keylist[indices[i]]:result[i]} for i in range(0,len(indices))]
print(bufferlist)
end = time.time()
print(end - start)

[{'46549': 294.9767}, {'83258': 295.26593}, {'55769': 300.30902}, {'83254': 301.18805}, {'52326': 301.47684}]
0.006722688674926758


In [16]:
import time
start = time.time()
j=0
bufferlist = [('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf)]
for distance in testrow:
    if distance < max(bufferlist,key=itemgetter(1))[1] and distance!=0:
        bufferlist.pop(bufferlist.index(max(bufferlist, key=itemgetter(1))))
        bufferlist.append((keylist[j],distance))
    j=j+1
print(bufferlist)
end = time.time()
print(end - start)

[('55769', 300.30902), ('46549', 294.9767), ('52326', 301.47684), ('83258', 295.26593), ('83254', 301.18805)]
0.05777549743652344


In [68]:
##run through distance matrix to determine top 5 most similar vectors
i=0
similar_vecs={}
for row in distances:
    indices = np.argsort(row)[1:6]
    result = row[indices]
    bufferlist = [(keylist[indices[j]],result[j]) for j in range(0,len(indices))]

    similar_vecs[keylist[i]] = bufferlist
    if i%100==0:
        print(i)
    i=i+1
    
    
#process output object for json serialization
processed_similar_vecs={}
for key,valuelist in similar_vecs.items():
    bufferlist=[{item[0]:float(item[1])} for item in valuelist]
    processed_similar_vecs[key]=bufferlist
    
    
#save as json
outfile = open('similar_vecs.json','w')
json.dump(processed_similar_vecs, outfile)

#save as pickle
outfile = open('similar_vecs.pickle','wb')
pickle.dump(similar_vecs, outfile)
outfile.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [15]:
##run through distance matrix to determine top 5 most similar vectors(for strict implementation)
i=0
similar_vecs={}
for row in distances2:
    indices = np.argsort(row)[1:6]
    result = row[indices]
    bufferlist = [(keylist[indices[j]],result[j]) for j in range(0,len(indices))]

    similar_vecs[keylist[i]] = bufferlist
    if i%100==0:
        print(i)
    i=i+1
    
    
#process output object for json serialization
processed_similar_vecs={}
for key,valuelist in similar_vecs.items():
    bufferlist=[{item[0]:float(item[1])} for item in valuelist]
    processed_similar_vecs[key]=bufferlist
    
    
#save as json
outfile = open('similar_vecs.json','w')
json.dump(processed_similar_vecs, outfile)

#save as pickle
outfile = open('similar_vecs.pickle','wb')
pickle.dump(similar_vecs, outfile)
outfile.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [16]:
infile = open('similar_vecs.json','r')
obj = json.load(infile)
# json.dump(processed_similar_vecs, outfile)

In [28]:
newdict = dict()
for key,value in obj.items():
    a = [[k for k,v in a.items()] for a in obj[key]]
    b = [item[0] for item in a]
    newdict[key]=b
    
    
with open('similar_vecs_final.json','w') as fp:
    json.dump(newdict,fp,indent=4)

In [27]:
a = [[k for k,v in a.items()] for a in obj['38727']]
b = [item[0] for item in a]
b

['50408', '50387', '57979', '98288', '48325']

In [26]:
obj['38727']

[{'50408': 298.1198812895652},
 {'50387': 310.39917468869413},
 {'57979': 316.4345451333185},
 {'98288': 319.2490664196342},
 {'48325': 319.40549923329064}]

In [75]:
result2[keylist.index('38727')]

array([-2.25237961e+02, -1.25230965e+02,  1.59898415e+01,  5.40582504e+01,
        1.05915634e+02, -3.33330250e+00, -6.74318619e+01, -4.16127434e+01,
       -2.80080624e+01,  6.97155533e+01, -1.01649605e+02,  1.73172169e+01,
        6.69426880e+01, -2.49627304e+01,  4.07382584e+01,  3.89820528e+00,
        4.30164185e+01, -1.17954338e+02, -1.15254784e+02,  4.93001518e+01,
        2.70924244e+01,  1.77661533e+01, -5.79419975e+01,  4.64620934e+01,
        6.19633904e+01,  1.40387278e+01, -1.45175781e+01,  7.83973389e+01,
       -7.82712021e+01,  1.31799765e+01,  1.26196518e+01, -4.26134186e+01,
       -2.90597401e+01,  1.23792858e+01, -1.74528770e+01,  1.59356651e+01,
       -6.46352768e+00,  3.71439896e+01, -1.45583677e+01,  2.87999744e+01,
       -5.95653915e+00, -2.55642376e+01, -4.57804260e+01, -7.97303867e+00,
        3.74092445e+01,  9.86425686e+00,  1.74372025e+01,  3.94931564e+01,
        1.66503048e+01,  8.12669830e+01, -2.92551727e+01, -3.73873672e+01,
       -2.13909950e+01,  

In [76]:
result2[keylist.index('85827')]

array([-1.29880157e+02, -6.24076042e+01, -2.22039719e+01,  4.27209930e+01,
       -3.01147385e+01, -2.95894642e+01,  2.64876747e+01, -3.76608047e+01,
        8.01295471e+01,  2.47239037e+01,  7.52463455e+01,  6.14964561e+01,
       -3.07298756e+01,  4.20371950e-01,  1.35464010e+01,  2.01339555e+00,
        1.61227665e+01, -1.33082247e+01, -8.92154408e+00, -7.53197403e+01,
       -1.49674261e+00, -1.17342024e+01, -1.43294783e+01,  1.27137318e+01,
       -8.48635025e+01,  1.21792936e+01,  2.23358822e+01, -2.68157902e+01,
        3.89964371e+01,  5.98705406e+01, -1.97756748e+01, -6.25628901e+00,
       -1.24046097e+01, -6.44364071e+00,  1.87733688e+01, -1.58568163e+01,
       -3.61831894e+01,  7.87512445e+00,  6.92125368e+00,  2.27518654e+01,
        1.15866985e+01,  3.68136063e+01, -9.18669033e+00,  2.33844051e+01,
        1.41259403e+01, -1.63739338e+01, -6.07696114e+01,  1.67615128e+01,
        1.50802958e+00,  5.28039885e+00, -1.22361240e+01, -8.60810089e+00,
       -3.26073990e+01, -

In [73]:
feature_flatten['38727']

array([0.       , 0.       , 0.       , ..., 0.       , 0.       ,
       4.2733884], dtype=float32)

In [ ]:
##run through distance matrix to determine top 5 most similar vectors
i=0
similar_vecs={}
for row in distances:
    j=0
    bufferlist = [('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf)]
    for distance in row:
        if distance < max(bufferlist,key=itemgetter(1))[1] and distance!=0:
            bufferlist.pop(bufferlist.index(max(bufferlist, key=itemgetter(1))))
            bufferlist.append((keylist[j],distance))
#         print(distance)
        j=j+1
#         break
    similar_vecs[keylist[i]] = bufferlist
#     print(row)
    if i%100==0:
        print(i)
    i=i+1
#     break



#process output object for json serialization
processed_similar_vecs={}
for key,valuelist in similar_vecs.items():
    bufferlist=[{item[0]:float(item[1])} for item in valuelist]
    processed_similar_vecs[key]=bufferlist
    
    
#save as json
outfile = open('similar_vecs.json','w')
json.dump(processed_similar_vecs, outfile)

#save as pickle
outfile = open('similar_vecs.pickle','wb')
pickle.dump(similar_vecs, outfile)
outfile.close()

In [96]:
# similarity_dict={}
# i=0
# # j=0
# for key1,vector1 in feature_flatten.items():
#     j=0
#     v1 = result2[i]
#     bufferlist = [('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf)]
#     #run through other pair and find 5 most similar vecs
#     for key2,vector2 in feature_flatten.items():
#         v2 = result2[j]
#         dst = distance.euclidean(v1,v2)
#         top5 = check_if_5_and_append(bufferlist,dst,key2)
#         j=j+1
#     similarity_dict[key1] = top5
#     if i%100 ==0:
#         print('we are on {}th vector'.format(i))
#     if i%1000 == 0:
#         print('we are on {}th vector'.format(i))
#         outfile = open('similar_pics/'+str(i)+'_similar_pics.json','w')
#         json.dump(similarity_dict,outfile)
#         outfile.close()
#         outfile = open('similar_pics/'+str(i)+'_similar_pics.json','w')
#         json.dump(similarity_dict,outfile)
#         outfile.close()
#     i=i+1
    

we are on 0th vector
we are on 0th vector
we are on 100th vector
we are on 200th vector
we are on 300th vector
we are on 400th vector


KeyboardInterrupt: 

In [108]:
#after PCA, calculate distance matrix
distances = euclidean_distances(result2, result2)

In [115]:
##run through distance matrix to determine top 5 most similar vectors
i=0
similar_vecs={}
for row in distances:
    j=0
    bufferlist = [('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf)]
    for distance in row:
        if distance < max(bufferlist,key=itemgetter(1))[1] and distance!=0:
            bufferlist.pop(bufferlist.index(max(bufferlist, key=itemgetter(1))))
            bufferlist.append((keylist[j],distance))
#         print(distance)
        j=j+1
#         break
    similar_vecs[keylist[i]] = bufferlist
#     print(row)
    if i%100==0:
        print(i)
    i=i+1
#     break

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [140]:
#process output object for json serialization
processed_similar_vecs={}
for key,valuelist in similar_vecs.items():
    bufferlist=[{item[0]:float(item[1])} for item in valuelist]
    processed_similar_vecs[key]=bufferlist

In [142]:
#save as json
outfile = open('similar_vecs.json','w')
json.dump(processed_similar_vecs, outfile)

In [124]:
#save as pickle
outfile = open('similar_vecs.pickle','wb')
pickle.dump(similar_vecs, outfile)
outfile.close()

In [26]:
keylist[0]##run through distance matrix to determine top 5 most similar vectors
i=0
similar_vecs={}
for row in distances:
    j=0
    bufferlist = [('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf),('1',math.inf)]
    for distance in row:
        if distance < max(bufferlist,key=itemgetter(1))[1] and distance!=0:
            bufferlist.pop(bufferlist.index(max(bufferlist, key=itemgetter(1))))
            bufferlist.append((keylist[j],distance))
#         print(distance)
        j=j+1
#         break
    similar_vecs[keylist[i]] = bufferlist
#     print(row)
    if i%100==0:
        print(i)
    i=i+1
#     break

'91341'